<a href="https://colab.research.google.com/github/tensorush/Phygitalism-3DML-Tasks/blob/master/Tasks/Task%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задачи

В некоторой архитектуре нейронной сети зависимость выхода $y \in \mathbb{R}$ от входа $(x_1, x_2) \in \mathbb{R}^2$ можно записать следующим образом:
$$
\begin{align*}
&f_1(x_1, x_2) = x_1 + x_2,\\
&f_2(x_1, x_2) = x_1 \cdot x_2,\\
&g_1(x_1, x_2) = \tan( f_1(x_1, x_2) + f_2(x_1, x_2) + 100),\\
&g_2(x_1, x_2) = f_1(x_1, x_2) \cdot f_2(x_1, x_2),\\
&y(x_1, x_2) = p(g_1(x_1,x_2), g_2(x_1, x_2)),
\end{align*}
$$
где $p(g_1, g_2)-$ некоторая неизвестная функция.

С помощью [механизма автоматического дифференцирования в PyTorch](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#) необходимо вычислить значения частных производных в разных точках:
$$
\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, 
$$

Для этого в файле `dev.json` приведены значения $\frac{\partial p}{\partial g_1}, \frac{\partial p}{\partial g_2}$ (обозначены как `dpdg1, dpdg2`), вычисленные при известных значениях входа. 

Нужно:

1. Загрузить данные из файла `dev.json`
2. Посчитать значения нужных производных для каждой точки.
3. Сохранить результат в формате CSV. Файл должен иметь следующую структуру:
```
id,dx1,dx2
....
```
`id` - это значение `id` из `dev.json`, `dx1, dx2`- значение производных, вычисленные в точке `id` из `dev.json.`

Ответ проверяется с помощью функции [math.isclose](https://docs.python.org/3/library/math.html#math.isclose) с точностью до `1e-8`. 

# Решение задачи

Импортируем `csv`, `json`, `math` и `torch` библиотеки.

In [ ]:
import csv
import json
import math
import torch

Скачаем и загрузим данные.

In [ ]:
!wget --quiet https://raw.githubusercontent.com/phygitalism/test-tasks-3dml/master/Task1/dev.json

with open('dev.json') as fin:
    data = json.load(fin)

Рассчитаем и сохраним результат для каждой точки в файл `results.csv`.

In [ ]:
with open('results.csv', 'a') as results:
    # Prepare to write the results
    writer = csv.writer(results)

    # Iterate over datapoints
    for point in data:
        # Read x1 and x2 into a single tensor x
        x = torch.DoubleTensor([point['x1'], point['x2']], requires_grad=True)

        # Read dpdg1 and dpdg2
        dpdg1 = point['dpdg1']
        dpdg2 = point['dpdg2']

        # Evaluate g1 and g2 at x
        f1 = x[0] + x[1]
        f2 = x[0] * x[1]
        g1 = torch.tan(f1 + f2 + 100)
        g2 = f1 * f2

        # Backpropagate gradients from g1 to x
        g1.backward(retain_graph=True)
        dg1dx1, dg1dx2 = x.grad.tolist()
        x.grad.zero_()

        # Backpropagate gradients from g2 to x
        g2.backward(retain_graph=True)
        dg2dx1, dg2dx2 = x.grad.tolist()
        x.grad.zero_()

        # Compute dydx1 and dydx2
        dydx1 = dpdg1 * dg1dx1 + dpdg2 * dg2dx1
        dydx2 = dpdg1 * dg1dx2 + dpdg2 * dg2dx2

        # Save the result
        writer.writerow([point['id'], dydx1, dydx2])

Выполним проверку результатов, аналитически вычислив производные:

$$
\begin{align*}
&\frac{\partial f_1}{\partial x_1} = 1,\\
&\frac{\partial f_1}{\partial x_2} = 1,\\
&\frac{\partial f_2}{\partial x_1} = x_2,\\
&\frac{\partial f_2}{\partial x_2} = x_1,\\
&\frac{\partial g_1}{\partial x_1} = \frac{\partial g_1}{\partial f_1}\frac{\partial f_1}{\partial x_1} + \frac{\partial g_1}{\partial f_2}\frac{\partial f_2}{\partial x_1} = \frac{1 + x_2}{\cos^2(x_1 + x_1x_2 + x_2 + 100)},\\
&\frac{\partial g_1}{\partial x_2} = \frac{\partial g_1}{\partial f_1}\frac{\partial f_1}{\partial x_2} + \frac{\partial g_1}{\partial f_2}\frac{\partial f_2}{\partial x_2} = \frac{1 + x_1}{\cos^2(x_1 + x_1x_2 + x_2 + 100)},\\
&\frac{\partial g_2}{\partial x_1} = \frac{\partial g_2}{\partial f_1}\frac{\partial f_1}{\partial x_1} + \frac{\partial g_2}{\partial f_2}\frac{\partial f_2}{\partial x_1} = x_2^2 + 2x_1x_2,\\
&\frac{\partial g_2}{\partial x_2} = \frac{\partial g_2}{\partial f_1}\frac{\partial f_1}{\partial x_2} + \frac{\partial g_2}{\partial f_2}\frac{\partial f_2}{\partial x_2} = x_1^2 + 2x_1x_2,\\
&\frac{\partial y}{\partial x_1} = \frac{\partial y}{\partial g_1}\frac{1 + x_2}{\cos^2(x_1 + x_1x_2 + x_2 + 100)} + \frac{\partial y}{\partial g_2}(x_2^2 + 2x_1x_2),\\
&\frac{\partial y}{\partial x_2} = \frac{\partial y}{\partial g_1}\frac{1 + x_1}{\cos^2(x_1 + x_1x_2 + x_2 + 100)} + \frac{\partial y}{\partial g_2}(x_1^2 + 2x_1x_2)
\end{align*}
$$

Рассчитаем и сохраним результат для каждой точки в файл `check_results.csv`.

In [ ]:
with open('check_results.csv', 'a') as check_results:
    # Prepare to write the results
    writer = csv.writer(check_results)

    # Iterate over all datapoints
    for point in data:
        # Read x1 and x2
        x1 = point['x1']
        x2 = point['x2']

        # Read dpdg1 and dpdg2
        dpdg1 = point['dpdg1']
        dpdg2 = point['dpdg2']

        # Compute dg1dx1 and dg1dx2
        dg1dx1 = (1 + x2) / (math.cos(x1 + x1 * x2 + x2 + 100) ** 2)
        dg1dx2 = (1 + x1) / (math.cos(x1 + x1 * x2 + x2 + 100) ** 2)

        # Compute dg2dx1 and dg2dx2
        dg2dx1 = (x2 ** 2 + 2 * x1 * x2)
        dg2dx2 = (x1 ** 2 + 2 * x1 * x2)

        # Compute dydx1 and dydx2
        dydx1 = dpdg1 * dg1dx1 + dpdg2 * dg2dx1
        dydx2 = dpdg1 * dg1dx2 + dpdg2 * dg2dx2

        # Save the result
        writer.writerow([point['id'], dydx1, dydx2])

Сверим полученные результаты.

In [ ]:
with open('results.csv', newline='') as results, open('check_results.csv', newline='') as check_results:
    # Prepare to read results
    reader = csv.reader(results, quoting=csv.QUOTE_NONNUMERIC)
    check_reader = csv.reader(check_results, quoting=csv.QUOTE_NONNUMERIC)

    # Compare all rows within 1e-8 tolerance
    for row, check_row in zip(reader, check_reader):
        if not math.isclose(row[1], check_row[1], rel_tol=1e-8):
            print(f'Row {row[0]} dy/dx1 do not match: {row[1]} != {check_row[1]}')
        if not math.isclose(row[2], check_row[2], rel_tol=1e-8):
            print(f'Row {row[0]} dy/dx2 do not match: {row[2]} != {check_row[2]}')
    print('Finished checking results!')

Finished checking results!
